In [1]:
import torch
import torch.nn as nn
from torchvision import models

# Load pretrained DenseNet121
model = models.densenet121(pretrained=True)

# Inspect model architecture
print(model)

# Check classifier layer
print("\nClassifier layer:", model.classifier)

# Get number of output features
num_ftrs = model.classifier.in_features
print("\nNumber of features in classifier:", num_ftrs)


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


100%|██████████| 30.8M/30.8M [00:00<00:00, 226MB/s]


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder("/content/drive/MyDrive/NN/dataset_split/train", transform=transform)
test_dataset = datasets.ImageFolder("/content/drive/MyDrive/NN/dataset_split/test", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [3]:
model.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(num_ftrs, 4)
)

import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()          # handles logits + labels
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader

# --------------------------
# 1. Device
# --------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --------------------------
# 2. Transforms
# --------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  # ImageNet mean
                         [0.229, 0.224, 0.225])  # ImageNet std
])

# --------------------------
# 3. Load dataset
# --------------------------
train_dataset = datasets.ImageFolder("/content/drive/MyDrive/NN/dataset_split/train", transform=transform)
test_dataset  = datasets.ImageFolder("/content/drive/MyDrive/NN/dataset_split/test", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

# --------------------------
# 4. Load DenseNet121
# --------------------------
model = models.densenet121(pretrained=True)

# Optional: freeze all DenseNet layers first
for param in model.parameters():
    param.requires_grad = False

# Replace classifier
num_ftrs = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(num_ftrs, 4)  # 4 classes
)
model = model.to(device)

# --------------------------
# 5. Loss and optimizer
# --------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=1e-4)  # only train classifier initially

# --------------------------
# 6. Training loop
# --------------------------
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        # Print per 50 batches
        if batch_idx % 50 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}] Batch [{batch_idx}/{len(train_loader)}] Loss: {loss.item():.4f}")

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}] Training Loss: {epoch_loss:.4f}")

    # --------------------------
    # 7. Validation per epoch
    # --------------------------
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}] Validation Accuracy: {acc:.2f}%\n")

# --------------------------
# 8. Optional: unfreeze top DenseNet layers for fine-tuning
# --------------------------
# for name, param in model.named_parameters():
#     if "denseblock4" in name or "norm5" in name:  # unfreeze last block
#         param.requires_grad = True
# optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch [1/10] Batch [0/264] Loss: 1.5445
Epoch [1/10] Batch [50/264] Loss: 1.3672
Epoch [1/10] Batch [100/264] Loss: 1.1518
Epoch [1/10] Batch [150/264] Loss: 1.0334
Epoch [1/10] Batch [200/264] Loss: 1.1254
Epoch [1/10] Batch [250/264] Loss: 1.0108
Epoch [1/10] Training Loss: 1.2089
Epoch [1/10] Validation Accuracy: 67.98%

Epoch [2/10] Batch [0/264] Loss: 0.9905
Epoch [2/10] Batch [50/264] Loss: 1.0839
Epoch [2/10] Batch [100/264] Loss: 0.8792
Epoch [2/10] Batch [150/264] Loss: 0.8418
Epoch [2/10] Batch [200/264] Loss: 0.9038
Epoch [2/10] Batch [250/264] Loss: 0.8296
Epoch [2/10] Training Loss: 0.8999
Epoch [2/10] Validation Accuracy: 75.31%

Epoch [3/10] Batch [0/264] Loss: 0.7060
Epoch [3/10] Batch [50/264] Loss: 0.8695
Epoch [3/10] Batch [100/264] Loss: 0.6935
Epoch [3/10] Batch [150/264] Loss: 0.8089
Epoch [3/10] Batch [200/264] Loss: 0.7944
Epoch [3/10] Batch [250/264] Loss: 0.7155
Epoch [3/10] Training Loss: 0.7589
Epoch [3/10] Validation Accuracy: 81.36%

Epoch [4/10] Batch [0/

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader

# --------------------------
# 1. Device
# --------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --------------------------
# 2. Transforms
# --------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# --------------------------
# 3. Load dataset
# --------------------------
train_dataset = datasets.ImageFolder("/content/drive/MyDrive/NN/dataset_split/train", transform=transform)
test_dataset  = datasets.ImageFolder("/content/drive/MyDrive/NN/dataset_split/test", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

# --------------------------
# 4. Load DenseNet121
# --------------------------
model = models.densenet121(pretrained=True)

# Freeze all layers initially
for param in model.parameters():
    param.requires_grad = False

# Replace classifier for 4 classes with dropout
num_ftrs = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(num_ftrs, 4)
)
model = model.to(device)

# --------------------------
# 5. Loss and optimizer (classifier only)
# --------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=1e-4)

# --------------------------
# 6. Initial training (classifier only)
# --------------------------
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        if batch_idx % 50 == 0:
            print(f"[Initial Train] Epoch {epoch+1}, Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}")

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"[Initial Train] Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    print(f"[Initial Train] Epoch {epoch+1}, Validation Accuracy: {acc:.2f}%\n")

# --------------------------
# 7. Fine-tuning last dense block
# --------------------------
for name, param in model.named_parameters():
    if "denseblock4" in name or "norm5" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

# Optimizer for fine-tuning with small learning rate
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)

# --------------------------
# 8. Fine-tuning training loop
# --------------------------
num_ft_epochs = 5
for epoch in range(num_ft_epochs):
    model.train()
    running_loss = 0.0

    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        if batch_idx % 50 == 0:
            print(f"[Fine-Tune] Epoch {epoch+1}, Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}")

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"[Fine-Tune] Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    print(f"[Fine-Tune] Epoch {epoch+1}, Validation Accuracy: {acc:.2f}%\n")


[Initial Train] Epoch 1, Batch 0/264, Loss: 1.4135
[Initial Train] Epoch 1, Batch 50/264, Loss: 1.1614
[Initial Train] Epoch 1, Batch 100/264, Loss: 1.1131
[Initial Train] Epoch 1, Batch 150/264, Loss: 1.2571
[Initial Train] Epoch 1, Batch 200/264, Loss: 0.9676
[Initial Train] Epoch 1, Batch 250/264, Loss: 0.9713
[Initial Train] Epoch 1, Loss: 1.1457
[Initial Train] Epoch 1, Validation Accuracy: 72.09%

[Initial Train] Epoch 2, Batch 0/264, Loss: 0.9379
[Initial Train] Epoch 2, Batch 50/264, Loss: 0.9193
[Initial Train] Epoch 2, Batch 100/264, Loss: 0.9397
[Initial Train] Epoch 2, Batch 150/264, Loss: 0.7482
[Initial Train] Epoch 2, Batch 200/264, Loss: 0.7745
[Initial Train] Epoch 2, Batch 250/264, Loss: 0.7142
[Initial Train] Epoch 2, Loss: 0.8622
[Initial Train] Epoch 2, Validation Accuracy: 79.52%

[Initial Train] Epoch 3, Batch 0/264, Loss: 0.8951
[Initial Train] Epoch 3, Batch 50/264, Loss: 0.6852
[Initial Train] Epoch 3, Batch 100/264, Loss: 0.7673
[Initial Train] Epoch 3, Batch

In [11]:
# --------------------------
# Continue fine-tuning
# --------------------------
num_more_epochs = 5  # or 3
model.train()  # make sure model is in training mode

# Optimizer remains the same (for last dense block + classifier)
# If you restarted, re-create optimizer like this:
# optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)

for epoch in range(num_more_epochs):
    running_loss = 0.0
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        if batch_idx % 50 == 0:
            print(f"[Continue FT] Epoch {epoch+1}, Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}")

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"[Continue FT] Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

    # Validation accuracy
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    print(f"[Continue FT] Epoch {epoch+1}, Validation Accuracy: {acc:.2f}%\n")
    model.train()  # back to training mode


[Continue FT] Epoch 1, Batch 0/264, Loss: 0.2302
[Continue FT] Epoch 1, Batch 50/264, Loss: 0.2756
[Continue FT] Epoch 1, Batch 100/264, Loss: 0.1377
[Continue FT] Epoch 1, Batch 150/264, Loss: 0.2404
[Continue FT] Epoch 1, Batch 200/264, Loss: 0.2587
[Continue FT] Epoch 1, Batch 250/264, Loss: 0.4292
[Continue FT] Epoch 1, Loss: 0.2363
[Continue FT] Epoch 1, Validation Accuracy: 92.86%

[Continue FT] Epoch 2, Batch 0/264, Loss: 0.0930
[Continue FT] Epoch 2, Batch 50/264, Loss: 0.2162
[Continue FT] Epoch 2, Batch 100/264, Loss: 0.3190
[Continue FT] Epoch 2, Batch 150/264, Loss: 0.1743
[Continue FT] Epoch 2, Batch 200/264, Loss: 0.7245
[Continue FT] Epoch 2, Batch 250/264, Loss: 0.1365
[Continue FT] Epoch 2, Loss: 0.2164
[Continue FT] Epoch 2, Validation Accuracy: 93.71%

[Continue FT] Epoch 3, Batch 0/264, Loss: 0.1418
[Continue FT] Epoch 3, Batch 50/264, Loss: 0.1740
[Continue FT] Epoch 3, Batch 100/264, Loss: 0.3522
[Continue FT] Epoch 3, Batch 150/264, Loss: 0.1807
[Continue FT] Epo

In [15]:
# --------------------------
# Continue fine-tuning
# --------------------------
num_more_epochs = 3  # or 3
model.train()  # make sure model is in training mode

# Optimizer remains the same (for last dense block + classifier)
# If you restarted, re-create optimizer like this:
# optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)

for epoch in range(num_more_epochs):
    running_loss = 0.0
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        if batch_idx % 50 == 0:
            print(f"[Continue FT] Epoch {epoch+1}, Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}")

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"[Continue FT] Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

    # Validation accuracy
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    print(f"[Continue FT] Epoch {epoch+1}, Validation Accuracy: {acc:.2f}%\n")
    model.train()  # back to training mode


[Continue FT] Epoch 1, Batch 0/264, Loss: 0.1777
[Continue FT] Epoch 1, Batch 50/264, Loss: 0.0506
[Continue FT] Epoch 1, Batch 100/264, Loss: 0.3216
[Continue FT] Epoch 1, Batch 150/264, Loss: 0.2303
[Continue FT] Epoch 1, Batch 200/264, Loss: 0.1147
[Continue FT] Epoch 1, Batch 250/264, Loss: 0.0882
[Continue FT] Epoch 1, Loss: 0.1423
[Continue FT] Epoch 1, Validation Accuracy: 95.22%

[Continue FT] Epoch 2, Batch 0/264, Loss: 0.2080
[Continue FT] Epoch 2, Batch 50/264, Loss: 0.2356
[Continue FT] Epoch 2, Batch 100/264, Loss: 0.1724
[Continue FT] Epoch 2, Batch 150/264, Loss: 0.1624
[Continue FT] Epoch 2, Batch 200/264, Loss: 0.1230
[Continue FT] Epoch 2, Batch 250/264, Loss: 0.2286
[Continue FT] Epoch 2, Loss: 0.1313
[Continue FT] Epoch 2, Validation Accuracy: 95.41%

[Continue FT] Epoch 3, Batch 0/264, Loss: 0.0352
[Continue FT] Epoch 3, Batch 50/264, Loss: 0.1027
[Continue FT] Epoch 3, Batch 100/264, Loss: 0.1196
[Continue FT] Epoch 3, Batch 150/264, Loss: 0.3032
[Continue FT] Epo

In [16]:
# Save the model weights
save_path = '/content/drive/MyDrive/NN/densenet_mri_ft.pth'
torch.save(model.state_dict(), save_path)
print(f"Model weights saved to: {save_path}")

Model weights saved to: /content/drive/MyDrive/NN/densenet_mri_ft.pth


In [17]:
model.eval()  # set model to evaluation mode
val_loss = 0.0
total_samples = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        outputs = model(images)
        loss = criterion(outputs, labels)  # CrossEntropyLoss
        val_loss += loss.item() * images.size(0)
        total_samples += images.size(0)

val_loss /= total_samples
print(f"Validation Loss for final model: {val_loss:.4f}")


Validation Loss for final model: 0.1369


In [ ]:
model.load_state_dict(torch.load(save_path))
model.to(device)
model.eval()  # for inference


In [18]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Compute metrics
precision = precision_score(all_labels, all_preds, average=None)  # per class
recall = recall_score(all_labels, all_preds, average=None)
f1 = f1_score(all_labels, all_preds, average=None)
conf_mat = confusion_matrix(all_labels, all_preds)

print("Per-class Precision:", precision)
print("Per-class Recall:", recall)
print("Per-class F1-score:", f1)
print("Confusion Matrix:\n", conf_mat)

# Optional: macro/micro averages
precision_macro = precision_score(all_labels, all_preds, average='macro')
recall_macro = recall_score(all_labels, all_preds, average='macro')
f1_macro = f1_score(all_labels, all_preds, average='macro')
print(f"Macro Precision: {precision_macro:.4f}, Macro Recall: {recall_macro:.4f}, Macro F1: {f1_macro:.4f}")


Per-class Precision: [0.98340249 0.90644491 0.96111111 0.96545455]
Per-class Recall: [0.94673768 0.92963753 0.98295455 0.9797048 ]
Per-class F1-score: [0.96472185 0.91789474 0.97191011 0.97252747]
Confusion Matrix:
 [[711  30   6   4]
 [ 10 436   8  15]
 [  1   5 346   0]
 [  1  10   0 531]]
Macro Precision: 0.9541, Macro Recall: 0.9598, Macro F1: 0.9568
